## Importing libraries and Loading data

In [1]:
import math, datetime, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import math, datetime, time, random
import matplotlib.pyplot as plt
import itertools
import tqdm

def load_data(data_path):
    data = pd.read_csv(data_path)  
    return data

inbound = load_data("inbound_loads.csv")
outbound = load_data("outbound_laods.csv")
weather = load_data("weather.csv")
#For loop to ensure that all pallet data is in the same dataframe
pallet = load_data("Pallet_history_Gold_Spike[0].csv")
for x in range(1, 10):
    pallet = pd.concat([pallet, load_data(f"Pallet_history_Gold_Spike[{x}].csv")])
pallet = pallet.drop(['lot_code', 
                      'tran_type', 
                      'final_pallet_code', 
                      'warehouse_facility_id',
                      'source_system_id'], axis=1)
trainentest = load_data("demand_kWtrain_val.csv")
train = trainentest.iloc[:273988,:]
test = trainentest.iloc[273988:, :]

# Preprocessing the Data
### 1. Load features and get them into the base dataframe

- load all features into the base dataframe
- create base_df

### 2. Preprocessing of features
- create dummy
- normalization (DECISION: Min-Max Normalization) 
- one hot encoding
- interpolating (fill in in between values)

### 3. Cutting off the dataframe 
DECISION: ? 
Which datapoints to keep?
Pallet_movement_5min starts 2nd of Jan 2019, so first rows is NaN. 

-  Demand values we have:
12/31/2018  21:15:00  upandincluding 10/11/2021/ 6:07:00
-  Predicting the demand from 
10/11/2021/ 6:08 upandincluding 12/13/2021  17:59:00 

### 1. Load features and get them into the base dataframe
#### Load features

Total weight of pallets coming into the warehouse in the previous 1h, 5h, 10h, 23h

In [2]:
###Incoming weight feature preprocessing
#load all data
incoming_weight_1h = load_data('incoming_weight_df_1h.csv')
incoming_weight_5h = load_data('incoming_weight_df_5h.csv')
incoming_weight_10h = load_data('incoming_weight_df_10h.csv')
incoming_weight_23h = load_data('incoming_weight_df.csv')
#rename columns 
incoming_weight_5h = incoming_weight_5h.rename(columns = {'weight_23h' : 'weight_5h'})
incoming_weight_1h = incoming_weight_1h.rename(columns = {'weight_23h' : 'weight_1h'})
incoming_weight_10h = incoming_weight_10h.rename(columns = {'weight_23h' : 'weight_10h'})
#get them to a datetime object
incoming_weight_1h['datetime_local'] = pd.to_datetime(incoming_weight_1h['datetime_local'])
incoming_weight_5h['datetime_local'] = pd.to_datetime(incoming_weight_5h['datetime_local'])
incoming_weight_10h['datetime_local'] = pd.to_datetime(incoming_weight_10h['datetime_local'])
incoming_weight_23h['datetime_local'] = pd.to_datetime(incoming_weight_23h['datetime_local'])
#set index to be datetime
incoming_weight_1h.set_index('datetime_local', inplace=True)
incoming_weight_5h.set_index('datetime_local', inplace=True)
incoming_weight_10h.set_index('datetime_local', inplace=True)
incoming_weight_23h.set_index('datetime_local', inplace=True)
#reshape them to start at 2018-31-12 9:15PM
incoming_weight_1h = incoming_weight_1h[2361:]
incoming_weight_5h = incoming_weight_5h[2330:] 
incoming_weight_10h = incoming_weight_10h[2326:]
incoming_weight_23h = incoming_weight_23h[2323:]
#Drop duplicates
incoming_weight_1h = incoming_weight_1h.loc[~incoming_weight_1h.index.duplicated()]
incoming_weight_5h = incoming_weight_5h.loc[~incoming_weight_5h.index.duplicated()]
incoming_weight_10h = incoming_weight_10h.loc[~incoming_weight_10h.index.duplicated()]
incoming_weight_23h = incoming_weight_23h.loc[~incoming_weight_23h.index.duplicated()]


Total number of pallets moved within 5 minutes

In [3]:
#load data
pallet_move_5min = load_data('pallet_movement_5min_ft.csv')
#rename column
pallet_move_5min.rename(columns = {'quantity' : 'pallet_movement_5min'}, inplace = True)
#make index a datetime object and set as index
pallet_move_5min['datetime_local'] = pd.to_datetime(pallet_move_5min['datetime_local'])
pallet_move_5min.set_index('datetime_local', inplace = True)
#delete duplicates
pallet_move_5min = pallet_move_5min.loc[~pallet_move_5min.index.duplicated()]

In [4]:
base_door = load_data("feature_inbound_outbound_door_open.csv")
base_door = base_door.rename(columns = {'total' : 'doors_open'})
base_door['datetime_local'] = pd.to_datetime(base_door['datetime_local'])
base_door.set_index('datetime_local', inplace=True)

#### Create the base dataframe base_df

In [5]:
def addtimecol(df, colname): ####input df and colname 
    df[colname] = pd.to_datetime(df[colname])         
    df['year'] = df[colname].dt.year
    df['month'] = df[colname].dt.month
    df['weekday'] = df[colname].dt.weekday
    df['day'] = df[colname].dt.day
    df['hour'] = df[colname].dt.hour
    df['minute'] = df[colname].dt.minute        
    return df

#Create new dummy dfs
base_df = train.copy()
base_weather = weather.copy()

#Remove unnecessary columns
base_df = base_df.drop('Unnamed: 0', axis=1)
base_weather = base_weather.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
base_weather["localstrptime"]= pd.to_datetime(base_weather["localstrptime"])
base_df['datetime_local'] = pd.to_datetime(base_df['datetime_local'])
base_weather = base_weather.rename(columns={'localstrptime':'datetime_local'})

#Add time columns.
addtimecol(base_df, 'datetime_local')

#Set index to datetime
base_df.set_index('datetime_local', inplace=True)
base_weather.set_index('datetime_local', inplace=True)

#Concatenate the weather DataFrame to the base DataFrame
base_df = pd.concat([base_df, base_weather], axis=1)

#Concatenate the incoming weight dataframe with the base dataframe
base_df = pd.concat([base_df, incoming_weight_1h], axis=1)
base_df = pd.concat([base_df, incoming_weight_5h], axis=1)
base_df = pd.concat([base_df, incoming_weight_10h], axis=1)
base_df = pd.concat([base_df, incoming_weight_23h], axis=1)

#Concatenate the pallet movement feature
base_df = pd.concat([base_df, pallet_move_5min], axis = 1)

#Concatenate the door feature
base_df = pd.concat([base_df, base_door], axis=1)

C:\Users\Rob Pennekamp\AppData\Local\Temp\ipykernel_14088\1540807240.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  base_df['datetime_local'] = pd.to_datetime(base_df['datetime_local'])


### 2. Preprocessing of features

#### Creating the dummy df with which we will do model selection




In [6]:
new_df = base_df.reindex(pd.date_range(start=base_df.index.min(),
                                                  end=base_df.index.max(),
                                                  freq='1min'))
new_df = new_df.drop('hour', axis=1)
print(new_df.columns)

Index(['demand_kW', 'year', 'month', 'weekday', 'day', 'minute', 'datetime',
       'Relative Humidity', 'Temperature', 'datetime_UTC', 'weight_1h',
       'weight_5h', 'weight_10h', 'weight_23h', 'pallet_movement_5min',
       'doors_open'],
      dtype='object')


In [7]:
def normalize_column(df, columnname):
    """Function which returns a Dataframe where the given column is normalized through min-max normalization."""
    df[f'{columnname}_normalized'] = (df[columnname] - df[columnname].min()) / (df[columnname].max() - df[columnname].min())
    return df.drop([columnname], axis=1)

def add_one_hot_encoder(df, colname):
    """
    Function which returns a DataFrame where the given column has been removed and replaced by
    one-hot-encoding columns for each value in the original column.
    """
    onehot = pd.get_dummies(df[colname], prefix=colname)
    return df.drop(colname, axis=1).join(onehot)

def interpolate_column(df, colname):
    df[f'{colname}_interpolated'] = df[colname].interpolate(method='linear')
    return df.drop([colname], axis=1)

In [8]:
interpolate = ['Temperature', 'Relative Humidity', 'demand_kW']
to_normalize = ['Relative Humidity_interpolated', 'Temperature_interpolated', 'weight_23h','weight_10h', 'weight_5h', 'weight_1h', 'pallet_movement_5min', 'doors_open']
add_one_hot_encoding = ['weekday', 'year', 'month']


for x in interpolate:
    new_df = interpolate_column(new_df, x)
for x in to_normalize:
    new_df = normalize_column(new_df, x)
for x in add_one_hot_encoding:
    new_df = add_one_hot_encoder(new_df, x)

In [13]:
#dropping = ['day', 'datetime', 'datetime_UTC', 'minute']
#for x in dropping:
#    new_df = new_df.drop(x, axis=1)
for x in new_df.columns:
    print(x, new_df[x].isna().sum())

demand_kW 0
Relative Humidity_interpolated_normalized 0
Temperature_interpolated_normalized 0
weight_23h_normalized 0
weight_10h_normalized 0
weight_5h_normalized 0
weight_1h_normalized 0
pallet_movement_5min_normalized 0
doors_open_normalized 0
weekday_0.0 0
weekday_1.0 0
weekday_2.0 0
weekday_3.0 0
weekday_4.0 0
weekday_5.0 0
weekday_6.0 0
year_2018.0 0
year_2019.0 0
year_2020.0 0
year_2021.0 0
month_1.0 0
month_2.0 0
month_3.0 0
month_4.0 0
month_5.0 0
month_6.0 0
month_7.0 0
month_8.0 0
month_9.0 0
month_10.0 0
month_11.0 0
month_12.0 0


In [10]:
new_df.rename(columns={'demand_kW_interpolated':'demand_kW'}, inplace=True)
new_df = new_df.dropna(subset = ['demand_kW', 'weight_1h_normalized', 'doors_open_normalized', 'pallet_movement_5min_normalized'], axis=0)

In [11]:
for x in new_df.columns:
    print(x, new_df[x].isna().sum())

demand_kW 0
Relative Humidity_interpolated_normalized 0
Temperature_interpolated_normalized 0
weight_23h_normalized 0
weight_10h_normalized 0
weight_5h_normalized 0
weight_1h_normalized 0
pallet_movement_5min_normalized 0
doors_open_normalized 0
weekday_0.0 0
weekday_1.0 0
weekday_2.0 0
weekday_3.0 0
weekday_4.0 0
weekday_5.0 0
weekday_6.0 0
year_2018.0 0
year_2019.0 0
year_2020.0 0
year_2021.0 0
month_1.0 0
month_2.0 0
month_3.0 0
month_4.0 0
month_5.0 0
month_6.0 0
month_7.0 0
month_8.0 0
month_9.0 0
month_10.0 0
month_11.0 0
month_12.0 0


In [ ]:
parameter_settings = {
    'n_estimators':[50, 100]
}
from sklearn.ensemble import RandomForestRegressor
from model_features import find_optimal_parameters
find_optimal_parameters(new_df.resample('1D').last(), RandomForestRegressor, parameter_settings, n_splits=8, verbose=True)

Testing model Model 1 with parameters: {'n_estimators': 50}
Starting fold 1
Training model...
Predicting...
Calculating rmse's...
Fold 1 train error: 94.20077421444279. Test error: 336.8260721742446. Time taken: 0.06571698188781738 s.
Starting fold 2
Training model...
Predicting...
Calculating rmse's...
Fold 2 train error: 87.31568393864218. Test error: 279.6037165178157. Time taken: 0.09008193016052246 s.
Starting fold 3
Training model...


In [ ]:
#fin_optimal_parameters(dummy_normalized_df, RandomForestRegressor, parameter_settings, n_splits=2, verbose=True)
from model_features import find_optimal_parameters
from sklearn.neural_network import MLPRegressor

parameter_settings = {
    'activation' : ['relu', 'tanh']
}

find_optimal_parameters(new_df, MLPRegressor, parameter_settings, n_splits=2, verbose=True)

In [108]:
dummy_df = base_df.copy()
#dummy_df = dummy_df.reset_index()
dummy_df = dummy_df.drop(['hour'], axis=1)
dummy_df = dummy_df.drop(['datetime'], axis=1)
dummy_df = dummy_df.drop(['datetime_local'], axis=1)
dummy_df = dummy_df.drop(['datetime_UTC'], axis=1)
dummy_df

dummy_normalized_df = dummy_df.copy()
#Still drop demand_kW NaNs
dummy_normalized_df = dummy_normalized_df.dropna(subset=['demand_kW'], axis=0)

In [109]:
dummy_df.pallet_movement_5min.count()

1592126

#### Preprocessing functions defined
DECISION: MinMax Normalization

#### interpolate/one-hot-encoding/normalizing
DECISION: ?  normalization of the doors_open

In [111]:
interpolate = ['Temperature', 'Relative Humidity', 'demand_kW']
to_normalize = ['Relative Humidity_interpolated', 'Temperature_interpolated', 'weight_23h','weight_10h', 'weight_5h', 'weight_1h', 'pallet_movement_5min', 'doors_open']
add_one_hot_encoding = ['weekday', 'year', 'month']


for x in interpolate:
    dummy_normalized_df = interpolate_column(dummy_normalized_df, x)
for x in to_normalize:
    dummy_normalized_df = normalize_column(dummy_normalized_df, x)
for x in add_one_hot_encoding:
    dummy_normalized_df = add_one_hot_encoder(dummy_normalized_df, x)

In [112]:
dummy_normalized_df.pallet_movement_5min_normalized.count()

273879

In [115]:
dummy_normalized_df = dummy_normalized_df.dropna(subset = ['pallet_movement_5min_normalized'])



# Model selection and building
## 1. Importing model libraries

## 2. Divide Train and Test data

## 3. Test models 

In [113]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

from sklearn.ensemble import RandomForestRegressor

## 2. Dividing train & test data

##### dummy_normalized_df['pallet_movement_5min'] drop NaN values

#### Model Testing functions


In [117]:
parameter_settings = {
    'n_estimators': [20],
    'criterion': ['squared_error', 'friedman_mse', 'squared_error', 'poisson'],
    #'max_depth': [2, 4, 6],
    #'min_samples_split': [2, 4, 8]
}


In [118]:
def model_optimizer(data, model, params, splits, verbose=False):
    """
    
    """
    parameter_combinations = itertools.product(*params.values())
    results = {}
    for parameters in parameter_combinations:
        start = time.time()
        params_dict = dict(zip(params.keys(), parameters))
        print('Testing model with parameters: ' + str(params_dict))
        current_model = model(**params_dict)
        mae, model = test_model(current_model, data, debug = False)
        print(f'Mean Absolute Error = {mae}')
        modelname = str(params_dict)
        end = time.time()
        print(f'Time spent: {end-start} seconds. \n')
        results[modelname] = [mae, (end-start), model]
    return results
        
#test = model_optimizer(data, RandomForestRegressor, parameter_settings)

# Functions to help with model testing

In [119]:
from sklearn.model_selection import TimeSeriesSplit
import time

def run_timeseries_model(data, model, n_splits = 5, verbose=False):
    """
    Function which receives a data and a model as input.
    Can also receive the amount of splits you want as input.
    For debugging purposes, set verbose to True.
    
    Outputs a list of:
        - The resulting model
        - The final RMSE
        - A list of the resulting Train RMSEs (per fold)
        - A list of the resulting Test RMSEs (per fold)
    """
    
    ts = TimeSeriesSplit(n_splits=n_splits)
    count = 1
    train_rmses = []
    test_rmses = []
    for train, test in ts.split(data):
        if verbose:
            print(f'Starting fold {count}')
        cv_train, cv_test = data.iloc[train], data.iloc[test]
        y_train = cv_train['demand_kW']
        x_train = cv_train.drop(['demand_kW'], axis=1)
        y_test = cv_test['demand_kW']
        x_test = cv_test.drop(['demand_kW'], axis=1)
        train_start = time.time()
        if verbose:
            print("Training model...")
        model.fit(x_train, y_train)
        train_stop = time.time()
        
        predict_start = time.time()
        if verbose:
            print("Predicting...")
        y_pred_test = model.predict(x_test)
        y_pred_train = model.predict(x_train)
        predict_stop = time.time()
        
        rmse_start = time.time()
        if verbose:
            print("Calculating rmse's...")
        train_rmse = mean_squared_error(y_train, y_pred_train, squared=False)
        test_rmse = mean_squared_error(y_test, y_pred_test, squared=False)
        rmse_stop = time.time()
        
        train_rmses.append(train_rmse)
        test_rmses.append(test_rmse)
        timings = [(train_stop - train_start), (predict_stop - predict_start), (rmse_stop - rmse_start), (rmse_stop - train_start)]
        if verbose:
            print(f'Fold {count} train error: {train_rmse}. Test error: {test_rmse}. Time taken: {timings[3]} s.')
        count += 1
    all_rmses = [train_rmses, test_rmses]
    return [model, test_rmse, all_rmses, timings]

In [120]:
import itertools
import pickle

def find_optimal_parameters(data, model, params, n_splits=5, verbose=False):
    """
    Function which will find the optimal parameters within set boudaries.
    Receives as input:
    - The data
    - A model
    - The dictionary of parameters to optimize:
        - Key: parameter name
        - Value: List of testing parameter values.
    - The amount of Timeseries splits for crossvalidation (Standard: 5)
    - Whether information should be printed or not (Standard: False)
    """
    
    parameter_combinations = itertools.product(*params.values())
    results = {}
    count = 1
    for x in parameter_combinations:
        model_name = f'Model {count}'
        results[model_name] = [dict(zip(params.keys(), x))]
        count +=1
    lowest = 1000000000
    best_model = ""
    print(results['Model 1'])
    for model_name in results.keys():
        params_dict = results[model_name][0]
        print(f'Testing model {model_name} with parameters: {params_dict}')
        current_model = model(**params_dict)
        results[model_name].append(run_timeseries_model(data, current_model, n_splits=n_splits, verbose=verbose))
        print(results[model_name])
        print(f'Finished testing model. Test RMSE: {results[model_name][1][1]}. Time taken: {results[model_name][1][3][3]}.')
        if results[model_name][1][1] < lowest:
            lowest = results[model_name][1][1]
            best_model = model_name
    print(f"Optimization completed. Best model is {best_model} with parameters {results[model_name]}. \n RMSE is {results[model_name][1][1]}.")
    pickle.dump(best_model, open("Best_model.pkl", 'wb'))
    return results

Testing model Model 1 with parameters: {'activation': 'relu'}
Starting fold 1
Training model...


C:\Users\Rob Pennekamp\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Predicting...
Calculating rmse's...
Fold 1 train error: 240.6029551160752. Test error: 308.79148242069664. Time taken: 450.47525000572205 s.
Starting fold 2
Training model...


C:\Users\Rob Pennekamp\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Predicting...
Calculating rmse's...
Fold 2 train error: 248.19592757147984. Test error: 495.5155389244764. Time taken: 918.9606564044952 s.
Finished testing model. Test RMSE: 495.5155389244764. Time taken: 1369.4359064102173.
Testing model Model 2 with parameters: {'activation': 'tanh'}
Starting fold 1
Training model...


...

Predicting...
Calculating rmse's...
Fold 1 train error: 219.2418440030109. Test error: 293.1655512748899. Time taken: 548.3995881080627 s.
Starting fold 2
Training model...


C:\Users\Rob Pennekamp\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Predicting...
Calculating rmse's...
Fold 2 train error: 230.40526142255104. Test error: 389.4801666143394. Time taken: 1287.4136629104614 s.
Finished testing model. Test RMSE: 389.4801666143394. Time taken: 1835.8132510185242.
Optimization completed. Best model is MLPRegressor(activation='tanh') with parameters 

 [{'activation': 'tanh'}, [MLPRegressor(activation='tanh'), 389.4801666143394, [[219.2418440030109, 230.40526142255104], [293.1655512748899, 389.4801666143394]], [548.3995881080627, 1287.4136629104614]]]. 

    RMSE is 389.4801666143394.
New best rmse reached! Pickling model.


C:\Users\Rob Pennekamp\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Rob Pennekamp\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'Model 1': [{'activation': 'relu'},
  [MLPRegressor(),
   495.5155389244764,
   [[240.6029551160752, 248.19592757147984],
    [308.79148242069664, 495.5155389244764]],
   [450.47525000572205, 918.9606564044952]]],
 'Model 2': [{'activation': 'tanh'},
  [MLPRegressor(activation='tanh'),
   389.4801666143394,
   [[219.2418440030109, 230.40526142255104],
    [293.1655512748899, 389.4801666143394]],
   [548.3995881080627, 1287.4136629104614]]]}

# Reducing DF Size

In [95]:
a1 = dummy_normalized_df[dummy_normalized_df[]].copy()
a1

,demand_kW,day,minute,Relative Humidity_interpolated_normalized,Temperature_interpolated_normalized,weight_23h_normalized,weight_10h_normalized,weight_5h_normalized,weight_1h_normalized,pallet_movement_5min_normalized,...,month_3.0,month_4.0,month_5.0,month_6.0,month_7.0,month_8.0,month_9.0,month_10.0,month_11.0,month_12.0
datetime_local,,,,,,,,,,,,,,,,,,,,,
2018-12-31 21:15:00,2064.101392,31.0,15.0,0.572375,0.442623,0.082728,0.110131,0.107216,0.000000,NaN,...,0,0,0,0,0,0,0,0,0,1
2018-12-31 21:30:00,1874.002081,31.0,30.0,0.572375,0.442623,0.082728,0.110131,0.068400,0.000000,NaN,...,0,0,0,0,0,0,0,0,0,1
2018-12-31 21:45:00,1988.168511,31.0,45.0,0.620183,0.426230,0.082728,0.109795,0.068400,0.000000,NaN,...,0,0,0,0,0,0,0,0,0,1
2018-12-31 22:00:00,2022.795943,31.0,0.0,0.620183,0.426230,0.082728,0.109795,0.047662,0.000000,NaN,...,0,0,0,0,0,0,0,0,0,1
2018-12-31 22:15:00,1986.981872,31.0,15.0,0.620183,0.426230,0.082728,0.109795,0.047662,0.000000,NaN,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-11 06:03:00,2217.847000,11.0,3.0,0.864746,0.540984,0.371198,0.190632,0.152500,0.015004,0.101205,...,0,0,0,0,0,0,0,1,0,0
2021-10-11 06:04:00,2184.012000,11.0,4.0,0.864746,0.540984,0.371198,0.190632,0.152500,0.015004,0.094917,...,0,0,0,0,0,0,0,1,0,0
2021-10-11 06:05:00,2159.482000,11.0,5.0,0.864746,0.540984,0.361868,0.190632,0.145594,0.015004,0.094618,...,0,0,0,0,0,0,0,1,0,0


In [100]:
a1.resample('15min').last()

,demand_kW,day,minute,Relative Humidity_interpolated_normalized,Temperature_interpolated_normalized,weight_23h_normalized,weight_10h_normalized,weight_5h_normalized,weight_1h_normalized,pallet_movement_5min_normalized,...,month_3.0,month_4.0,month_5.0,month_6.0,month_7.0,month_8.0,month_9.0,month_10.0,month_11.0,month_12.0
datetime_local,,,,,,,,,,,,,,,,,,,,,
2018-12-31 21:15:00,2064.101392,31.0,15.0,0.572375,0.442623,0.082728,0.110131,0.107216,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2018-12-31 21:30:00,1874.002081,31.0,30.0,0.572375,0.442623,0.082728,0.110131,0.068400,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2018-12-31 21:45:00,1988.168511,31.0,45.0,0.620183,0.426230,0.082728,0.109795,0.068400,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2018-12-31 22:00:00,2022.795943,31.0,0.0,0.620183,0.426230,0.082728,0.109795,0.047662,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2018-12-31 22:15:00,1986.981872,31.0,15.0,0.620183,0.426230,0.082728,0.109795,0.047662,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-11 05:00:00,2164.534000,11.0,14.0,0.864746,0.540984,0.363871,0.180877,0.159336,0.078976,0.031514,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2021-10-11 05:15:00,2215.498000,11.0,29.0,0.864746,0.540984,0.371198,0.190632,0.171300,0.048270,0.113407,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2021-10-11 05:30:00,1978.980000,11.0,44.0,0.804240,0.557377,0.371198,0.190632,0.152500,0.015004,0.054271,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
